## **Bank Customer Churn Prediction**

### **1. Introduction**

#### **1.1 Objectives**
1. Conduct data exploration and data cleaning
2. Conduct descriptive analysis of the data.
3. Determine relationship between variables.
4. Conduct churn prediction, given the variables in the dataset.
5. Share insights and findings through visualizations and observations.

#### **1.2 Methodology**
1. Use Python and necessary packages in conducting exploratory data analysis and data visualizations.
2. Check the dataset for incorrect datatypes and data.
3. Check the dataset for null values.
4. If there are incorrect datatypes and data and presence of null values, employ necessary actions to correct the data.
5. Create additional columns (if necessary) by using data extracted from an existing column.
6. Create data visualizations for descriptive analysis.
7. Plot relationship between variables using visualization tools such as Seaborn and Matplotlib.
8. Write observations based on the results of the analysis.
9. Create a machine learning model to predict the likelihood that a customer will churn, based on the variables in the dataset.
10. Evaluate the results of the machine learning model and share insights and recommendations.
